**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2023

# Uranium Mill Network Dynamics Simulation Report
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Students:** Kyle Rizzuto, Robert Townsend, Jacob Duprey, Kevin McMenemy <br>
**Course:** Engy-4390 Nuclear Systems Design and Analysis  <br>
**Mentor:** Prof. Valmor F. de Almeida

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**27Sep2023**

<font color="red">vfda: I strongly suggest you to have a project outline document with division of tasks and timeline for the members of the group. Who does what and what the deadlines are. What is the progress so far?</font>

---
## Table of Contents<a id="toc"></a>
* [Introduction](#intro)
* [Objectives](#obj)
* [Tools](#tools)
* [Task Timeline](#task)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Process Modules](#IPM)
  + [Leaching](#leach)#Robert
  + [Clarification/Filtration](#clear)#Kyle
  + [Solvent Extraction/Stripping](#solvent)#Jacob
  + [Precipitation](#precip)#Dr. Almeida
  + [Evaporation/Drying](#evap)#Kevin
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

## [Introduction](#toc)<a id="intro"></a>

<font color="red">vfda: This needs to be beefed up.</font>

&nbsp;&nbsp; Yellowcake is a material produced from uranium during the milling process. It is a mixture of different types of uranium oxide that combine to form a yellow tinted compound. This material is vital in the formation of reactor fuel in the later steps of the process known as uranium conversion. The two main processes used to recover uranium from the ground are In-Situ leaching and open-pit mining. In-Situ mining, which involves the injection of acid into the ground that dissolves uranium before being brought back to the surface in a pipe, is vastly more used in the US than open-pit because it allows a direct recovery of uranium without the need for digging out uranium bearing rock and moving it to a separate facility for processing. This does limit the places that the method can be used because the acid can contaminate groundwater if it is allowed to travel too deep. Once the uranium is pulled from the ground, it undergoes chemical separations to isolate the oxides.

&nbsp;&nbsp;The general unit operations that follow the mining of the ore are size reduction (crushing/grinding), then acid leaching. The acid leaching process uses acidic and alkaline aqueous solutions with agitation for four to 48 hours at ambient temperature. Usually sulfuric acid is used and results in a final liquor with a pH of 1.5. Manganese dioxide is typically then used with the liquor to result in [UO2(SO4)3]2- The resulting uranium-oxide containing ions must then be separated using either a solvent extraction or ion exchange. The ion exchange usually uses an ion-exchange resin along with any of a variety of processing equipment such as fixed or moving bed along with another chemical solution to “sorb” the complex ion from the leach solution. Solvent extraction involves using alkyl phosphoric acids and alkyl amides as solvents to remove the ions containing uranium from their leach solution. The uranium oxide must then be precipitated from the solutions created by the solvent extraction or ion exchange. The resulting “yellow cake” as it is commonly referred to is a mix of uranium oxides which is then shipped to a final purification facility.[4]

&nbsp;&nbsp;Yellow cake, which is about 90% U3O8, is then chemically processed into uranium tetrafluoride (UF4) through a process called hydrofluorination. This reaction mechanism consists of heating beds of solid uranium oxide in an environment of gaseous hydrofluoric acid and stripping away the oxide layers until only UF4 remains. HF is typically used in a 20% stoichiometric excess to maximize the conversion percentage of U3O8 to at least 98%. Before the uranium can be enriched however, the UF4 must be further converted into UF6, also known as uranium hexafluoride. This is because enrichment is largely dependent on gaseous centrifugal separation, and UF6 does not possess a liquid form under standard pressure conditions. Uranium fluorides are also the preferred form for enrichment because fluorine only exists in one natural isotope, meaning that the heavier molecules will always contain the Uranium-238.

&nbsp;&nbsp;&nbsp;&nbsp;Python code will be used to develop dynamic comprehensive systems that accurately represent the milling process as well as the complex parts of the plant that simulate a steady state reactor. This involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis. Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  

&nbsp;&nbsp;&nbsp;&nbsp;There are environmental and non-proliferation applications that come from developing a dynamic model simulating the steps of the uranium fuel cycle. For instance, disturbances to the system can be modeled to determine the impact on the immediate processes involved as well as the cascading effects on the next steps of the cycle. This information can lead to engineers designing risk mitigating and preventative measures that can make the system safer for operation and for the environment. This application is useful for the everchanging frontier of finite resource management. A major example is the White Mesa uranium mill in Utah, which already has a system in place where they take in low level radioactive waste from companies and reprocess it using their heap leaching treatment methods. If natural sources of uranium become depleted over the years, it is vital to understand and develop alternative methods of fuel production from nuclear waste as well as other uranium remnant sources. 

&nbsp;&nbsp;The White Mesa Uranium Mill in Utah is the sole producer of uranium oxide in the United States, and operates using a heap based leaching approach. For the purposes of this model, we have selected White Mesa as the basis for the creation of modules. Upon examination of the flowsheet diagram created by White Mesa available in a 1993 IAEA report on the state of uranium milling, we have elected to model the following five unit operations in the milling process:leaching, clarification, solvent extraction, precipitation, and drying. These operations effectively encapsulate the majority of losses and operating costs in the uranium milling process, and capture both the very first and final stages of raw ore preparation in the production of uranium oxide.[3]


<font color="red">Where is the flowsheet? Which plant? How about the documents that needed to be reviewed? IAEA document with plants?</font>

## [Objectives](#toc)<a id="obj"></a>

- Python code will be used to develop dynamic comprehensive systems that accurately represent the milling process as well as the complex parts of the plant that simulate a steady state reactor. This involves both mathematical models of the system’s material streams and visual representations of the data on a time dependent basis. 
- Modules to model the leaching, filtration, solvent extraction, precipitation, and evaporation steps will be developed.
- Deliverable content will be prepared using Jupyter notebooks to seamlessly integrate working code with descriptive markdown text.  
- A flowsheet based on White Mesa will be developed as the basis for our modules



## [Tools](#toc)<a id="tools"></a>

&nbsp;&nbsp;&nbsp;&nbsp;Github is an open-source repository site that allows users to share information publicly while still controlling who can change the published information. For this project, Dr. Almeida has administrative control over the changes published online. Each student has their own independently controlled version of the project paperwork that they can manipulate and update without affecting the official version. When a student makes a change, their individual version can then be submitted for review to Dr. Almeida through Github by using a ‘pull request’. Once the changes are accepted, each student can then resynchronize their personal versions to reflect any added information. 

&nbsp;&nbsp;&nbsp;&nbsp;Cortix is a library for Python that was created by Dr. Almeida and his research group to simplify the process of producing dynamic system models. The full open-source code can be found on Github, where anyone can download and use the library and its available unit operation modules. The Github repository also contains examples that use the code to simulate various chemical interactions while delivering time dependent data. 

&nbsp;&nbsp;&nbsp;&nbsp;Jupyter Notebook is also open-source, and it is a way to integrate ordinary text with multiple programming languages such as Python. Python is a widely used programming language that can be likened to a more simplified version of C. This is a popular language for dynamic modelling and has been in use for over thirty years. It is commonplace in many programs used in the nuclear industry, making it a great candidate for this project. 

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

-  First, a series of mass balances and component balances are made based on the desired input of 1 ton of raw material.
- These balances then take into account the chemical kinetics and related rates required in the operation of White Mesa's chemical reactions and fluid flow operations
- Once baseline models of each individual process exist, they are then programmed into 5 separate but connected modules within Cortix
- Each process will be capable of interacting with the modules connected to it on either end, allowing them to share data such as the incoming and outgoing streams from one process to the next

|     **Module**        |      **Inputs**      |      **Outputs**       |     **Legend**     |
|:----------------------|:---------------------|:-----------------------|:-------------------|
| Leaching             |        |           |        |
| Filtration             |        |           |        |
| Solvent Extraction             |        |           |        |
| Precipitation             |        |           |        |
| Evaporation            |        |           |        |

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

&nbsp;&nbsp;&nbsp;&nbsp;When this project was initially proposed, we were expecting to model an In-Situ style uranium mill. This is due to the advantage of causing less ground disturbance for deeper reservoirs than with a traditional open pit mine, as well as the ability to be a one site process due to the removal of the ore crushing and transportation steps of the process. Even though this process shows promising signs, there is not much publicly available information on the operations of an In-Situ mine. This is most likely due to it being a newer method of uranium recovery with proprietary knowledge not yet being shared. As a result, this presented a challenge of finding a better option that still has realistic and useful applications. This was what led us to the option of modelling White Mesa’s heap leaching method. 

&nbsp;&nbsp;&nbsp;&nbsp;Some of the advantages of modelling the White Mesa mill have already been mentioned. Being the only active uranium mill in the US makes it the only sensible option for a national based model. Groups interested in preserving and expanding our knowledge of the processes involved in the nuclear fuel cycle such as the Uranium Science and Technology Center in Oak Ridge, Tennessee will be receiving their domestic milled uranium from White Mesa. By using this mill as an example, the exact chemical interactions and properties involved will be consistent with the physical product being used throughout all future lab scale operations conducted in the US. 

## [Initialization of Process Modules](#toc)<a id="IPM"></a>

The Cortix model is ...

In [1]:
'''Setup a Cortix plant'''

from cortix import Cortix
from cortix import Network

plant = Cortix(use_mpi=False, splash=True) # System top level
plant_net = plant.network = Network() # Network

[4350] 2020-12-21 18:49:23,874 - cortix - INFO - Created Cortix object 
_____________________________________________________________________________
                             L A U N C H I N G                               
_____________________________________________________________________________
      ...                                        s       .     (TAAG Fraktur)
   xH88"`~ .x8X                                 :8      @88>
 :8888   .f"8888Hf        u.      .u    .      .88      %8P      uL   ..
:8888>  X8L  ^""`   ...ue888b   .d88B :@8c    :888ooo    .     .@88b  @88R
X8888  X888h        888R Y888r ="8888f8888r -*8888888  .@88u  ""Y888k/"*P
88888  !88888.      888R I888>   4888>"88"    8888    888E`    Y888L
88888   %88888      888R I888>   4888> "      8888      888E      8888
88888 `> `8888>     888R I888>   4888>        8888      888E      `888N
`8888L %  ?888   ! u8888cJ888   .d888L .+    .8888Lu=   888E   .u./"888&
 `8888  `-*""   /   "*888*P"    ^"8888*"     ^%8

### [Leaching](#toc)<a id="leach"></a>
The leaching step consists of treating the uranium ore with an alkaline or acid reagent to dissolve it. The uranium is dissolved in the solution through a reaction described by [UO<sub>3</sub> + 2H<sup>+</sup> --> UO<sub>2</sub><sup>2+</sup> + H<sub>2</sub>O]. In an acidic leaching solution the uranyl ion then combines with the sulphate ion, [UO<sub>2</sub><sup>2+</sup> + 3(SO<sub>4</sub>)<sup>2-</sup> --> UO<sub>2</sub>(SO<sub>4</sub>)<sub>3</sub><sup>4-</sup>] to form a stable complex. In an alkaline leaching solution the chemical reaction is [UO<sub>2</sub><sup>2+</sup> + 3(CO<sub>3</sub>)<sup>2-</sup> --> UO<sub>2</sub>(CO<sub>3</sub>)<sub>3</sub><sup>4-</sup>]. Acidic leaching is preferred over alkaline leaching where the ore contains less than 10% carbonates, typically sulfuric acid is used for acidic leaching and sodium carbonate or sodium bicarbonate is used for alkaline leaching. The oxidation reaction in acidic solutions is usually achieved by the ferric ion Fe<sup>3+</sup> acting as an electron transfer agent in the solution. In order to sustain this oxidation, pyrolosite, sodium chlorate, or Caro's acid can be used to regenerate the Fe<sup>3+</sup>. Alkaline leaching cannot sustain a high enough ferric ion concentration so higher pressure, temperature, leaching times, and a finer ore grind are needed. 

### [Clarification/Filtration](#toc)<a id="clear"></a>


&nbsp;&nbsp;&nbsp;&nbsp;Uranium filtration is the separation of solids from a slurry stream that allows liquid through pores. This step of the nuclear fuel cycle comes after the initial leaching of the raw ore and prepares the product stream by partially purifying it before it is sent to a solvent extraction device. If the presence of impurities reaches 200ppm it can lead to column plugging which can halt the solvent extraction process entirely, so the accepted practice is to keep the value at 10ppm or less. The unit operation involved is a vacuum drum filter with stacked filter leaves that continuously purify the feed liquid. Filterability of the feed is predetermined using a sample of the feed with a single filter leaf. The filtration rate of the bulk system is controlled via the use of induced vacuum through the drum or added pressure to the stream.[5]  

The first calculation involved in operation of the system is the form filtration rate: 


Z = K(1/𝜃)^0.5 Equation F.1  

Where Z is the form filtration rate in mass per unit area per hour as cake is being formed, θ is the formation time in hours, and K is a constant. The total time of a cycle through the drum is typically 3-6x the cake formation time. Typical inputs from the leaching process include soluble uranyl carbonate complex ion [UO2(CO3)3]4- dissolved in (typically) sulfuric acid. Impurities in the liquor are metals, such as vanadium, molybdenum, thorium, aluminum, and iron as well as sulfate ions.  

&nbsp;&nbsp;&nbsp;&nbsp;Due to the open pit and heap leaching process disturbing a large piece of the environment relative to what is extracted for fuel, most of the removed impurities are recovered as tailings. There are environmental requirements for the quality of tailings such as the maximum allowable ppm of harmful acids, therefore the contents of the tailings must be modelled both to account for the overall mass balance and the environmental requirements for safe operation. 

A pressure based constant rate filtration model has the form: 

ΔP = μαcqV/A^2+μRq/A  Equation F.2 
 

Where ΔP is the pressure drop over the cake and filter (Pa), μ is filtrate viscosity (Pa*s), α is the average specific resistance (m/kg), A is the filter area, q is the filtrate flow rate (m^3/s), V is filtrate volume (m^3), c is the average dry cake mass per unit volume of filtrate (kg/m^3), and R is the filter’s medium resistance (m^-1). Since there is a limit to the operating pressure in the system, the maximum pressure difference is the upper bound limit. The increase in resistance to filtration due to the buildup of cake is pressure dependent, as explained in Darcy’s law of cake filtration. 

### [Solvent Extraction](#toc)<a id="solvent"></a>

Solvent extraction is a method used to remove a solute from a solution. This is done by introducing the solution to an immiscible solvent. In this case, the solution will be the pregnant liquor from the leaching step with the solute being Uranium in the form of U<sub>3</sub>O<sub>8</sub>. The solvent in this case is an amine salt diluted in kerosene. The amine salt will extract the uranium ions in the liquor creating an organic complex that is insoluble in water. After several settling and decanting stages to remove the aqueous solution, we are now left with just the organic complex. This can be mixed with an inorganic salt solution, such as sodium chloride, to strip the uranium from the complex. The resulting product is yellow-cake which is ready to be fed into the evaporation stage.

### [Precipitation](#toc)<a id="precip"></a>

### [Evaporation/Drying](#toc)<a id="evap"></a>

## [Startup](#toc)<a id="su"></a>

## [Steady State](#toc)<a id="ss"></a>

## [Disturbance](#toc)<a id="malfunction"></a>

## [References](#toc)<a id="refs"></a>



- [1] V. F. de Almeida, [Cortix](https://cortix.org), *Network Dynamics Simulation*, University of Massachusetts Lowell, Dept. of Chemical Engineering (Nuclear program).
- [2] V. F. de Almeida, [*Engy-4350: Nuclear Reactor Engineering*](https://github.com/dpploy/engy-4350), University of Massachusetts Lowell, Dept. of Chemical Engineering (Nuclear Program).
- [3] “35 No. 9 Uranium Extraction Technology - IAEA.” Uranium Extraction Technology, IAEA, 1993, www.pub.iaea.org/MTCD/Publications/PDF/trs359_web.pdf. 
- [4] “NRC: Conventional Uranium Mills.” Www.nrc.gov, www.nrc.gov/materials/uranium-recovery/extraction-methods/conventional-mills.html.
- [5] “STATE OF UTAH RADIOACTIVE MATERIALS LICENSE No. UT1900479 .” White Mesa Uranium Mill - License Renewal, Denison Mines Corp., 2007, www.documents.deq.utah.gov/legacy/businesses/e/energy-fuels-resources-usa/docs/2014/index.html. 